# 2-1-Genome_annotation_TEs 
Manuel Jara-Espejo$^{1}$\
Aboobaker lab, Department of Biology, University of Oxford

## Contents of notebook
1. Introduction
2. Identification and annotation of repetitve elements in *Phaw5.1* assembly
3. Assign TE famyli ID to each annotated TE element 
4. TE distribution in Phaw5.1 genome

### 1. Introduction

### 2. Identification and annotation of repetitve elements in *Phaw5.1* assembly

In [ ]:
%%bash
#cd /drives/ssd1/manuel/phaw/2022_analysis/annotation/repetetive_elements/

#Build database
/drives/raid/AboobakerLab/software/RepeatModeler-2.0.2a/BuildDatabase -name phaw_sambaAsm.DB -engine rmblast /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/change_scafNames/phaw_sambaAsm.scaff_seqs_editedScafNames.fa

#Run RepeatModeler
nohup /drives/raid/AboobakerLab/software/RepeatModeler-2.0.2a/RepeatModeler -database phaw_sambaAsm.DB -engine ncbi -pa 16 > repeatModeler.out 2>&1 & 

#Run Repeat Masking 
nohup /drives/raid/AboobakerLab/software/RepeatMasker/RepeatMasker -dir . -pa 16 -gff -lib ./RM_317837.MonJul41113502022/consensi.fa.classified  /drives/ssd1/manuel/phaw/2022_analysis/phaw_gapfilling/TGS-GapCloser_anlysis/results_sambaAsm/change_scafNames/phaw_sambaAsm.scaff_seqs_editedScafNames.fa > repeatMasking.out 2>&1 &

In [1]:
%%bash

#Phaw5.1 repeatome - Output table with basic stats
cat /drives/raid/AboobakerLab/manuel/data_phaw_analysis/repeats_gapFilledAsm/phaw_sambaAsm.scaff_seqs_editedScafNames.fa.tbl

file name: phaw_sambaAsm.scaff_seqs_editedScafNames.fa
sequences:        280585
total length: 2823710058 bp  (2731003535 bp excl N/X-runs)
GC level:         41.27 %
bases masked: 1596874276 bp ( 56.55 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
Retroelements       692645    358214746 bp   12.69 %
   SINEs:                0            0 bp    0.00 %
   Penelope         111526     45187488 bp    1.60 %
   LINEs:           644713    332658947 bp   11.78 %
    CRE/SLACS          738       477092 bp    0.02 %
     L2/CR1/Rex     339488    193383413 bp    6.85 %
     R1/LOA/Jockey   40331     21033146 bp    0.74 %
     R2/R4/NeSL      10895      8043685 bp    0.28 %
     RTE/Bov-B       16202      7741164 bp    0.27 %
     L1/CIN4             0            0 bp    0.00 %
   LTR elements:     47932     25555799 bp    0.91 %
     BEL/Pao         12642      7527752 bp    0.27 %
     Ty1/

In [3]:
%%bash 
#Phaw5.1 repeatome - Output GTF annotation file
cat /drives/raid/AboobakerLab/manuel/data_phaw_analysis/repeats_gapFilledAsm/phaw_sambaAsm.scaff_seqs_editedScafNames.fa.out.gff | head -10

##gff-version 2
##date 2022-07-05
##sequence-region phaw_sambaAsm.scaff_seqs_editedScafNames.fa
Scaffold_100000_HRSCAF_104010	RepeatMasker	similarity	2	158	10.2	+	.	Target "Motif:rnd-5_family-509" 1133 1293
Scaffold_100000_HRSCAF_104010	RepeatMasker	similarity	155	512	 3.6	-	.	Target "Motif:rnd-1_family-797" 98 455
Scaffold_100001_HRSCAF_104011	RepeatMasker	similarity	7	338	28.1	+	.	Target "Motif:rnd-1_family-425" 451 776
Scaffold_100001_HRSCAF_104011	RepeatMasker	similarity	336	400	 0.0	+	.	Target "Motif:rnd-1_family-866" 338 402
Scaffold_100001_HRSCAF_104011	RepeatMasker	similarity	400	512	 2.7	-	.	Target "Motif:rnd-1_family-608" 3645 3756
Scaffold_100002_HRSCAF_104012	RepeatMasker	similarity	1	39	 2.6	+	.	Target "Motif:rnd-1_family-560" 221 259
Scaffold_100002_HRSCAF_104012	RepeatMasker	similarity	44	257	 8.9	+	.	Target "Motif:rnd-1_family-867" 1 226


### 3. Assign TE famyli ID to each annotated TE element 

In [3]:
%%bash

#1. Use getTEn_familyRM.rb to get a table containing:
#    TE number, assigned using getTEn_familyRM.rb script (i.e TE_1)
#    TE family, assigned using RepeatMasking (i.e rnd-5_family-638 )
#    Output file TEnumber_TE_RMid.txt, containing 5690059 out of 5935576 TE assigned to families

cat /drives/ssd1/manuel/phaw/2022_analysis/te_analysis/te_expression/getTEn_familyRM.rb 

count= 1

File.open("/drives/raid/AboobakerLab/manuel/data_phaw_analysis/repeats_gapFilledAsm/phaw_sambaAsm.scaff_seqs_editedScafNames.fa.out.gff","r").each do |line|
    if line[0] != "#"
        te_id = "TE_#{count}" #Simple TE id, used later to merge
        target_id = line.chomp.split("\t")[8] #Broad familiy  ID assigned during annotation.
        
#Since I used SalmonTE to quantify expression, I changed the ID identifier to the oned generated by the tool: scaff+start+end (range=Scaffold_100000_HRSCAF_104010:1-158)
        
        te_id = line.chomp.split("\t")[0] + ":" + (line.chomp.split("\t")[3].to_i - 1).to_s + ":" + line.chomp.split("\t")[4]

        puts target_id.split("\"")[1].gsub("Motif:","") + "\t" + te_id
        count +=1
    end
end

In [6]:
%%bash
head -5 /drives/ssd1/manuel/phaw/2022_analysis/te_analysis/te_expression/TEnumber_TE_RMid.txt

rnd-5_family-509	Scaffold_100000_HRSCAF_104010:1:158
rnd-1_family-797	Scaffold_100000_HRSCAF_104010:154:512
rnd-1_family-425	Scaffold_100001_HRSCAF_104011:6:338
rnd-1_family-866	Scaffold_100001_HRSCAF_104011:335:400
rnd-1_family-608	Scaffold_100001_HRSCAF_104011:399:512


In [8]:
%%bash 
#2. Use getfamilyRM_Class.rb to get a table containing:
#    TE family, assigned using RepeatMasking (i.e rnd-5_family-638)
#    TE family assigned from RepBase (i.e LINE)
#    Output TE_RMid_RepBase.txt, 3848 families. 3130 are unknown families
cat /drives/ssd1/manuel/phaw/2022_analysis/te_analysis/te_expression/getfamilyRM_Class.rb 

count= 1

File.open("/drives/raid/AboobakerLab/manuel/data_phaw_analysis/repeats_gapFilledAsm/phaw_sambaAsm.DB-families.fa","r").each do |line|
    
    #Use onlye fasta seqeunces header to get info: 
    #>rnd-1_family-321#LINE/L2 ( RepeatScout Family Size = 297, Final Multiple Alignment Size = 100, Localized to 3474 out of 4163     contigs )
    
    if line[0] == ">"
        puts line.split[0].split("#")[0][1..-1] + "\t" + line.split[0].split("#")[1]
    end
end

#phaw_sambaAsm.DB-families.fa contains consensus sequences for each family annotated (RepeatMasking), as well as their putative homologous family in RepBase

In [12]:
%%bash
#3. Merge both ouput files based on TE_id using:
#    join <(sort TEnumber_TE_RMid.txt) <(sort TE_RMid_RepBase.txt) -t $'\t' > TE_Id_Family_full.txt. 

#Not all TE_ids belonged to families annotated based on RepDB, that's why the final set of TE is slightly shorter (5690059 out of 5935576)
echo -e "RepeatMasker_Id\tMyId\tRepBaseId" 
head -5 /drives/ssd1/manuel/phaw/2022_analysis/te_analysis/te_expression/TE_Id_Family_full.txt

RepeatMasker_Id	MyId	RepBaseId
rnd-1_family-0	Scaffold_100013_HRSCAF_104023:346:374	Unknown
rnd-1_family-0	Scaffold_100032_HRSCAF_104042:216:512	Unknown
rnd-1_family-0	Scaffold_100414_HRSCAF_104425:136:229	Unknown
rnd-1_family-0	Scaffold_100_HRSCAF_734:43181:43270	Unknown
rnd-1_family-0	Scaffold_101355_HRSCAF_105366:263:507	Unknown


### 4. TE distribution in Phaw5.1 genome

##### 3.1 Intersect TE annotetion with gene annotation files. Output will be bed files containing exonic-, intronic- and intergeniC-TEs.

In [16]:
%%bash
#gff2bed < /drives/raid/AboobakerLab/manuel/data_phaw_analysis/repeats_gapFilledAsm/phaw_sambaAsm.scaff_seqs_editedScafNames.fa.out.gff > phaw_sambaAms_TEs.bed
head -5  /drives/ssd1/manuel/phaw/2022_analysis/te_analysis/te_methylation/phaw_sambaAms_TEs.bed

Scaffold_100000_HRSCAF_104010	1	158	.	10.2	+	RepeatMasker	similarity	.	Target "Motif:rnd-5_family-509" 1133 1293
Scaffold_100000_HRSCAF_104010	154	512	.	 3.6	-	RepeatMasker	similarity	.	Target "Motif:rnd-1_family-797" 98 455
Scaffold_100001_HRSCAF_104011	6	338	.	28.1	+	RepeatMasker	similarity	.	Target "Motif:rnd-1_family-425" 451 776
Scaffold_100001_HRSCAF_104011	335	400	.	 0.0	+	RepeatMasker	similarity	.	Target "Motif:rnd-1_family-866" 338 402
Scaffold_100001_HRSCAF_104011	399	512	.	 2.7	-	RepeatMasker	similarity	.	Target "Motif:rnd-1_family-608" 3645 3756


##### 3.2 Identify feature-specific TEs:

In [18]:
%%bash

#Intronic:
#bedtools intersect -wa -f 0.75 -s -a phaw_sambaAms_TEs.bed -b /drives/ssd1/manuel/phaw/2022_analysis/mzt_analysis/bin_coverage_analysis/transcripts_MultiExon_exon.bed > exonicTEs.bed

#Exonic:
#bedtools intersect -wa -f 0.75 -s -a phaw_sambaAms_TEs.bed -b /drives/ssd1/manuel/phaw/2022_analysis/mzt_analysis/bin_coverage_analysis/transcripts_MultiExon_intron.bed > intronicTEs.bed

#Intergenic:
#bedtools intersect -wa -wb -f 0.75 -v -s -a phaw_sambaAms_TEs.bed  -b /drives/ssd1/manuel/phaw/2022_analysis/mzt_analysis/bin_coverage_analysis/unigenes_transcripts_MultiExon.bed | cut -f 1,2,3,4,5,6,10,14 > intergenicTEs.bed

head -5  /drives/ssd1/manuel/phaw/2022_analysis/te_analysis/te_methylation/intergenicTEs.bed

Scaffold_100000_HRSCAF_104010	1	158	.	10.2	+	Target "Motif:rnd-5_family-509" 1133 1293
Scaffold_100000_HRSCAF_104010	154	512	.	 3.6	-	Target "Motif:rnd-1_family-797" 98 455
Scaffold_100001_HRSCAF_104011	6	338	.	28.1	+	Target "Motif:rnd-1_family-425" 451 776
Scaffold_100001_HRSCAF_104011	335	400	.	 0.0	+	Target "Motif:rnd-1_family-866" 338 402
Scaffold_100001_HRSCAF_104011	399	512	.	 2.7	-	Target "Motif:rnd-1_family-608" 3645 3756
